In [1]:
import glob
import re
import pdfplumber
import pandas as pd
import os
from path import Path

# Define keyword groups
keywords_groups = {
    '101-1':'Reporting Principles',
'101-2':'Using the GRI Standards for  sustainability reporting',
'101-3':'Making claims related to the use of the GRI Standards',
'102-2':'Activities, brands, products, and services',
'102-3':'Location of headquarters',
'102-4':'Location of operations',
'102-5':'Ownership and legal form',
'102-6':'Markets served',
'102-7':'Scale of the organization',
'102-8':'Information on employees and other workers',
'102-9':'Supply chain',
'102-10':'Significant changes to the organization and its supply chain',
'102-11':'Precautionary Principle or approach',
'102-12':'External initiatives',
'102-13':'Membership of associations',
'102-14':'Statement from senior decision-maker',
'102-15':'Key impacts, risks, and opportunities',
'102-16':'Values, principles, standards, and norms of behavior',
'102-17':'Mechanisms for advice and concerns about ethics',
'102-18':'Governance structure',
'102-19':'Delegating authority',
'102-20':'Executive-level responsibility for economic, environmental,  and social topics ',
'102-21':'Consulting stakeholders on economic, environmental,  and social topics ',
'102-22':'Composition of the highest governance body and its committees ',
'102-23':'Chair of the highest governance body',
'102-24':'Nominating and selecting the highest governance body',
'102-25':'Conflicts of interest',
'102-26':'Role of highest governance body in setting purpose, values,  and strategy',
'102-27':'Collective knowledge of highest governance body',
'102-28':'Evaluating the highest governance body’s performance',
'102-29':'Identifying and managing economic, environmental, and social impacts',
'102-30':'Effectiveness of risk management processes',
'102-31':'Review of economic, environmental, and social topics',
'102-32':'Highest governance body’s role in sustainability reporting',
'102-33':'Communicating critical concerns',
'102-34':'Nature and total number of critical concerns',
'102-35':'Remuneration policies',
'102-36':'Process for determining remuneration',
'102-37':'Stakeholders’ involvement in remuneration',
'102-38':'Annual total compensation ratio',
'102-39':'Percentage increase in annual total compensation ratio',
'102-40':'List of stakeholder groups',
'102-41':'Collective bargaining agreements',
'102-42':'Identifying and selecting stakeholders',
'102-43':'Approach to stakeholder engagement',
'102-44':'Key topics and concerns raised',
'102-45':'Entities included in the consolidated financial statements ',
'102-46':'Defining report content and topic Boundaries',
'102-47':'List of material topics',
'102-48':'Restatement of information',
'102-49':'Changes in reporting',
'102-50':'Reporting period',
'102-51':'Date of most recent report',
'102-52':'Reporting cycle',
'102-53':'Contact point for questions regarding the report',
'102-54':'Claims of reporting in accordance with the GRI Standards',
'102-55':'GRI content index',
'102-56':'External assurance ',
'103-1':'Explanation of the material topic and its Boundary',
'103-2':'The management approach and its components ',
'103-3':'Evaluation of the management approach',
'201-1':'Direct economic value generated and distributed',
'201-2':'Financial implications and other risks and opportunities  due to climate change',
'201-3':'Defined benefit plan obligations and other retirement plans',
'201-4':'Financial assistance received from government',
'202-1':'Ratios of standard entry level wage by gender compared  to local minimum wage',
'202-2':'Proportion of senior management hired from the  local community',
'203-1':'Infrastructure investments and services supported',
'203-2':'Significant indirect economic impacts',
'204-1':'Proportion of spending on local suppliers',
'205-1':'Operations assessed for risks related to corruption',
'205-2':'Communication and training about anti-corruption policies  and procedures ',
'205-3':'Confirmed incidents of corruption and actions taken ',
'206-1':'Legal actions for anti-competitive behavior, anti-trust,  and monopoly practices ',
'301-1':'Materials used by weight or volume',
'301-2':'Recycled input materials used',
'301-3':'Reclaimed products and their packaging materials',
'302-1':'Energy consumption within the organization',
'302-2':'Energy consumption outside of the organization',
'302-3':'Energy intensity',
'302-4':'Reduction of energy consumption',
'302-5':'Reduction in energy requirements of products and services',
'303-1':'Interactions with water as a shared resource',
'303-2':'Management of water discharge-related impacts',
'303-3':'Water withdrawal',
'303-4':'Water discharge',
'303-5':'Water consumption',
'304-1':'Operational sites owned, leased, managed in, or adjacent  to, protected areas and areas of high biodiversity value  outside protected areas',
'304-2':'Significant impacts of activities, products, and services  on biodiversity',
'304-3':'Habitats protected or restored',
'304-4':'IUCN Red List species and national conservation list  species with habitats in areas affected by operations',
'305-1':'Direct (Scope 1) GHG emissions',
'305-2':'Energy indirect (Scope 2) GHG emissions',
'305-3':'Other indirect (Scope 3) GHG emissions',
'305-4':'GHG emissions intensity',
'305-5':'Reduction of GHG emissions',
'305-6':'Emissions of ozone-depleting substances (ODS)',
'305-7':'Nitrogen oxides (NOX), sulfur oxides (SOX), and other  significant air emissions',
'306-1':'Water discharge by quality and destination',
'306-2':'Waste by type and disposal method',
'306-3':'Significant spills',
'306-4':'Transport of hazardous waste',
'306-5':'Water bodies affected by water discharges and/or runoff',
'307-1':'Non-compliance with environmental laws and regulations',
'308-1':'New suppliers that were screened using environmental criteria ',
'308-2':'Negative environmental impacts in the supply chain and  actions taken',
'401-1':'New employee hires and employee turnover',
'401-2':'Benefits provided to full-time employees that are not provided  to temporary or part-time employees',
'401-3':'Parental leave',
'402-1':'Minimum notice periods regarding operational changes',
'403-1':'Occupational health and safety management system',
'403-2':'Hazard identification, risk assessment, and incident investigation ',
'403-3':'Occupational health services',
'403-4':'Worker participation, consultation, and communication on occupational health and safety',
'403-5':'Worker training on occupational health and safety',
'403-6':'Promotion of worker health ',
'403-7':'Prevention and mitigation of occupational health and safety impacts directly linked by business relationships',
'403-8':'Workers covered by an occupational health and safety management system',
'403-9':'Work-related injuries',
'403-10':'Work-related ill health',
'404-1':'Average hours of training per year per employe',
'404-2':'Programs for upgrading employee skills and transition  assistance programs',
'404-3':'Percentage of employees receiving regular performance  and career development reviews',
'405-1':'Diversity of governance bodies and employees',
'405-2':'Ratio of basic salary and remuneration of women to men ',
'406-1':'Incidents of discrimination and corrective actions taken',
'407-1':'Operations and suppliers in which the right to freedom  of association and collective bargaining may be at risk',
'408-1':'Operations and suppliers at significant risk for incidents  of child labor',
'409-1':'Operations and suppliers at significant risk for incidents  of forced or compulsory labor ',
'410-1':'Security personnel trained in human rights policies or procedures',
'411-1':'Incidents of violations involving rights of indigenous peoples',
'412-1':'Operations that have been subject to human rights reviews  or impact assessments',
'412-2':'Employee training on human rights policies or procedures ',
'412-3':'Significant investment agreements and contracts that  include human rights clauses or that underwent human  rights screening',
'413-1':'Operations with local community engagement, impact assessments, and development programs',
'413-2':'Operations with significant actual and potential negative impacts on local communities',
'414-1':'New suppliers that were screened using social criteria',
'414-2':'Negative social impacts in the supply chain and actions taken',
'415-1':'Political contributions ',
'416-1':'Assessment of the health and safety impacts of product and service categories',
'416-2':'Incidents of non-compliance concerning the health and safety impacts of products and service',
'417-1':'Requirements for product and service information and labeling',
'417-2':'Incidents of non-compliance concerning product and service information and labeling',
'417-3':'Incidents of non-compliance concerning  marketing communications',
'418-1':'Substantiated complaints concerning breaches of customer  privacy and losses of customer data',
'419-1':'Non-compliance with laws and regulations in the social  and economic area',
'102-1':'Name of the organization'
}

# Function to remove non-printable characters
def remove_illegal_chars(text):
    cleaned_text = re.sub(r'[\x00-\x1f\x7f-\x9f]', '', text)
    return cleaned_text

# Function to search for keywords in PDF files using pdfplumber
def search_keywords_in_pdf_plumber(file_path):
    file_name = file_path.split("/")[-1]
    results = []
    keyword_found = False
    try:
        with pdfplumber.open(file_path) as pdf:
            for page_num, page in enumerate(pdf.pages, start=1):
                text = page.extract_text()
                if text:
                    text = remove_illegal_chars(text)  # Clean the text
                    for keyword, group in keywords_groups.items():
                        if re.search(r'\b' + re.escape(keyword) + r'\b', text, re.IGNORECASE):
                            results.append({
                                "Nama PDF": file_name,
                                "Keyword": keyword,
                                "Page": page_num,
                                "Group": group,
                                "Content": text
                            })
                            keyword_found = True
    except Exception as e:
        results.append({
            "Nama PDF": file_name,
            "Error": "Error processing PDF: " + str(e),
            "Keyword": "",
            "Page": "",
            "Group": "",
            "Content": ""
        })
    if not keyword_found and not results:
        results.append({
            "Nama PDF": file_name,
            "Keyword": "",
            "Page": "",
            "Group": "",
            "Content": ""
        })
    return results

folder_tahun = ['2021']
folder_isi = ['Annual Report','Integrated Report','Sustainability Report']

# folder_tahun = ['Singapore-2021']
# folder_isi = ['Annual Report','Integrated Report','Sustainability Report']

# Path ke folder utama yang berisi subfolder-subfolder
main_folder_path = Path("D:/CESGS/Scrap")
output_folder_path = Path("D:/CESGS/Scrap")

# Mengiterasi setiap nama dalam list untuk memproses file txt
for t in folder_tahun:
    # Folder path
    folder_path = main_folder_path/f"{t}"
    # Menentukan path folder "[P] ISI" untuk setiap nama
    output_fp = output_folder_path / f"[O] {t}"
    # Membuat folder jika belum ada
    # Membuat folder jika belum ada
    if not output_fp.exists():
        output_fp.mkdir()
    # output_fp.mkdir()
    for i in folder_isi:
        all_results = []
        folder_pdf = folder_path/i
        excel_path = output_fp / f"GS 16.1_{i}.xlsx"
        # Load existing results if Excel file already exists to avoid reprocessing
        if os.path.exists(excel_path):
            df_existing = pd.read_excel(excel_path)
            processed_files = df_existing["Nama PDF"].unique()
        else:
            df_existing = pd.DataFrame()
            processed_files = []

        # Assume a directory 'Singapore-2022/Annual Report/' contains PDF files
        pdf_files = glob.glob(f"{folder_pdf}/*.pdf")
        for file_path in pdf_files:
            file_name = file_path.split("/")[-1]
            if file_name not in processed_files:  # Check if file has not been processed yet
                results = search_keywords_in_pdf_plumber(file_path)
                df_results = pd.DataFrame(results)
                df_existing = pd.concat([df_existing, df_results], ignore_index=True)

                df_existing.to_excel(excel_path, index=False)  # Save after each file is processed
                print(f"{file_name} processed")
print("All files have been processed.")